In [ ]:
import json
import pandas as pd

In [ ]:
with open('../../data/meta_Electronics.jsonl', 'r') as f:
    first_line= json.loads(f.readline())

In [ ]:
first_line

## filter items that have been obserserved for the first time after 2022

In [ ]:
def filter_data(data: dict)-> dict:
    """Filter data based on some criteria"""
    filter = False
    if int(data['details']['Date First Available'][-4:]) < 2022:
        filter = True
    return filter

In [ ]:
with open('../../data/meta_Electronics.jsonl','r') as fp:
    with open('../../data/meta_Electronics_2022_2023.jsonl','a',encoding="utf-8") as fp_out:
        with open('../../data/meta_Electronics_2022_2023_no_date.jsonl','a',encoding="utf-8") as fp_out_no_date:
            i = 0
            for line in fp:
                try:
                    data = json.loads(line.strip())
                    filter = filter_data(data)
                    if not filter:
                        json.dump(data, fp_out)
                        fp_out.write('\n')
                        fp_out.flush()
                except Exception as e:
                    json.dump(data, fp_out_no_date)
                    fp_out_no_date.write('\n')
                    fp_out_no_date.flush()
                i += 1
                if i % 10000 == 0:
                     print(f"Processed {i} lines")

In [ ]:
def filter_catergory(data:dict) -> dict:
    """Filter the data based on the category"""
    if data['main_category'] == None:
        return True
    return False

### Split the items into two categories: "has main category", "does not have main category"

In [ ]:
with open('../../data/meta_Electronics_2022_2023.jsonl','r') as fp:
    with open('../../data/meta_Electronics_2022_2023_with_catergory.jsonl','a', encoding='utf-8') as fp_out:
        with open('../../data/meta_Electronics_2022_2023_with_catergory.jsonl', 'a', encoding='utf-8') as fp_out_no_category:
            i = 0
            for line in fp:
                data = json.loads(line.strip())
                try:
                    filter = filter_catergory(data)
                    if not filter:
                        json.dump(data, fp_out)
                        fp_out.write('\n')
                        fp_out.flush()
                except Exception as e:
                    json.dump(data, fp_out_no_category)
                    fp_out_no_category.write('\n')
                    fp_out_no_category.flush()
                i += 1
                if i % 100000 == 0:
                    print(f'Processed {i} lines')

## Explore distribution by categories

In [ ]:
df = pd.read_json('../../data/meta_Electronics_2022_2023_with_catergory.jsonl',lines=True)

In [ ]:
df.head()

In [ ]:
df['main_category'].value_counts().plot(kind='bar')

## Filter out items that have at least 100 ratings

In [ ]:
df_ratings_100 = df[df['rating_number'] > 100]

In [ ]:
len(df)

In [ ]:
len(df_ratings_100)

In [ ]:
df_ratings_100['main_category'].value_counts().plot(kind='bar')

## Explore distribution of ratings

In [ ]:
df_ratings_100['average_rating'].plot(kind='hist', bins=50, range=(0, 5))

## Sample 1000 items

In [ ]:
df_sample_1000 = df_ratings_100.sample(n=1000, random_state=20)

In [ ]:
df_sample_1000['average_rating'].plot(kind='hist', bins=50, range=(0, 5))

In [ ]:
df_sample_1000['price'].plot(kind='hist', bins=100, range=(0, 500))

In [ ]:
df_sample_1000['main_category'].value_counts().plot(kind='bar')

In [ ]:
df_ratings_100.to_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100.jsonl", orient='records', lines=True)

In [ ]:
df_sample_1000.to_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", orient='records', lines=True)

## Extract ratings that match sampled data.

In [ ]:
df_ratings_100 = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100.jsonl", lines=True)
df_sample_1000 = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [ ]:
with open("../../data/Electronics.jsonl", 'r') as fp:
    with open("../../data/Electornics_2022_2023_with_category_ratings_100.jsonl", 'a') as fp_out:
        id_list = set(df_ratings_100['parent_asin'].values)
        i = 0
        for line in fp:
            data = json.loads(line.strip())
            if data['parent_asin'] in id_list:
                json.dump(data, fp_out)
                fp_out.write('\n')
                fp_out.flush()
            i += 1
            if i % 100000 == 0:
                print(f"Processed {i} lines")

In [ ]:
with open("../../data/Electornics_2022_2023_with_category_ratings_100.jsonl", 'r') as fp:
    with open("../../data/Electornics_2022_2023_with_category_ratings_100_sample_1000.jsonl", 'a') as fp_out:
        id_list = set(df_sample_1000['parent_asin'].values)
        i = 0
        for line in fp:
            data = json.loads(line.strip())
            if data['parent_asin'] in id_list:
                json.dump(data, fp_out)
                fp_out.write('\n')
                fp_out.flush()
            i += 1
            if i % 100000 == 0:
                print(f"Processed {i} lines")